# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [44]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
from bokeh.plotting import show
import holoviews as hv
import cartopy as ct
import geoviews as gv


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Port-Aux-Francais,-49.3500,70.2167,38.98,67,70,36.01,TF,broken clouds
1,1,Sabang,5.8933,95.3214,82.36,80,99,17.69,ID,overcast clouds
2,2,Blackmans Bay,-43.0167,147.3167,42.53,78,27,6.80,AU,scattered clouds
3,3,Saudarkrokur,65.7461,-19.6394,46.31,81,100,3.67,IS,overcast clouds
4,4,Manali,13.1667,80.2667,91.49,78,100,11.23,IN,overcast clouds


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Create a map of the cities with the size of the point corresponding to the humidity
city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    hover_cols=["City"],
    alpha=0.7,
    geo=True,
    tiles="CartoLight",
    title="City Humidity",
    colorbar=True
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter for ideal temperature and humidity conditions
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80) & (city_data_df["Humidity"] < 50) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
15,15,Klyuchi,52.2667,79.1667,79.16,22,0,12.33,RU,clear sky
58,58,Chu,43.5983,73.7614,72.82,17,0,13.38,KZ,clear sky
59,59,Moyynkum,44.2914,72.9469,75.90,13,0,14.65,KZ,clear sky
87,87,Serdar,38.9764,56.2757,79.92,19,0,14.79,TM,clear sky
134,134,Troitskoye,52.9821,84.6757,75.70,27,0,10.09,RU,clear sky
156,156,Khanty-Mansiysk,61.0042,69.0019,73.51,23,0,13.42,RU,clear sky
187,187,Qingdao,36.0986,120.3719,73.00,28,0,13.42,CN,clear sky
295,295,Belyy Yar,53.6039,91.3903,75.09,20,0,11.18,RU,clear sky
424,424,Luwingu,-10.2621,29.9271,74.98,44,0,6.17,ZM,clear sky
446,446,Melilla,35.2937,-2.9383,79.00,44,0,14.97,ES,clear sky


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Port-Aux-Francais,TF,-49.3500,70.2167,67,
1,Sabang,ID,5.8933,95.3214,80,
2,Blackmans Bay,AU,-43.0167,147.3167,78,
3,Saudarkrokur,IS,65.7461,-19.6394,81,
4,Manali,IN,13.1667,80.2667,78,
...,...,...,...,...,...,...
577,Puerto Carreno,CO,6.1890,-67.4859,78,
578,San Luis De La Loma,MX,17.2706,-100.8939,74,
579,Ciudad Lazaro Cardenas,MX,17.9583,-102.2000,86,
580,Benavente,ES,42.0025,-5.6783,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
params = {"categories": "accommodation.hotel",
    "limit": 1,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},10000"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
       
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address.json()["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
Port-Aux-Francais - nearest hotel: Keravel
Sabang - nearest hotel: Sabang Hill
Blackmans Bay - nearest hotel: Villa Howden
Saudarkrokur - nearest hotel: Mikligarður
Manali - nearest hotel: Vijay Park
Waitangi - nearest hotel: Hotel Chathams
Hawaiian Paradise Park - nearest hotel: No hotel found
Igarka - nearest hotel: Заполярье
Tazovsky - nearest hotel: Тазовчанка
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Grytviken - nearest hotel: No hotel found
College - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
Adamstown - nearest hotel: No hotel found
Tazacorte - nearest hotel: App Leyma
Harlingen - nearest hotel: Holiday Inn Express
Klyuchi - nearest hotel: No hotel found
Conceicao Do Araguaia - nearest hotel: No hotel found
Yangjiang - nearest hotel: Jinghu Hotel
Usogorsk - nearest hotel: Единственная
Burayevo - nearest hotel: No hotel found
Vorgashor - nearest hotel: София
Bartica - nearest hotel: Rockville Hotel
Daru - nearest hotel: Kuki
K

Hobyo - nearest hotel: No hotel found
Orthez - nearest hotel: Kyriad
Kokopo - nearest hotel: Hotel Kokopo
Mukalla - nearest hotel: فندق الجابري
Maintirano - nearest hotel: Hotel Safari
Geraldton - nearest hotel: Ocean Centre Hotel
Vila Velha - nearest hotel: Hotel Prainha
Hit - nearest hotel: No hotel found
Aasiaat - nearest hotel: SØMA
Karratha - nearest hotel: Karratha International Hotel
Illapel - nearest hotel: Domingo Ortiz de Rosas
Betio Village - nearest hotel: Seaman Hostel
Acapulco De Juarez - nearest hotel: Hotel del Valle
Menongue - nearest hotel: No hotel found
San Juan De Miraflores - nearest hotel: Venue Hospedaje
Antofagasta - nearest hotel: Hotel Colón
St. John'S - nearest hotel: Alt Hotel
Udachny - nearest hotel: Вилюй
Duba - nearest hotel: No hotel found
Kulhudhuffushi - nearest hotel: No hotel found
Megion - nearest hotel: Адрия
Okhotsk - nearest hotel: No hotel found
Brazzaville - nearest hotel: BLISS HOTEL
Ajdovscina - nearest hotel: Pigal
Yako - nearest hotel: No 

Hailin - nearest hotel: No hotel found
Grand Falls-Windsor - nearest hotel: Hotel Robin Hood
Ati - nearest hotel: No hotel found
Luganville - nearest hotel: Beachfront Resort
Lashio - nearest hotel: Golden Kaniri Hotel
Panuco - nearest hotel: Hotel Plaza
Slave Lake - nearest hotel: Northwest Inn
Afaahiti - nearest hotel: Omati Lodge
Pachino - nearest hotel: Hotel Celeste
Portland - nearest hotel: the Hoxton
Richards Bay - nearest hotel: Protea Hotel
Mokshan - nearest hotel: No hotel found
Susuman - nearest hotel: No hotel found
Lindi - nearest hotel: Baraka inn
Poggiardo - nearest hotel: Tesoretto
Minzhu - nearest hotel: No hotel found
Xinyuan - nearest hotel: No hotel found
Ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
Saint-Joseph - nearest hotel: PALM Hotel & Spa
Salahonda - nearest hotel: No hotel found
Sao Felix Do Xingu - nearest hotel: Hotel Terraço
Raduzhnyy - nearest hotel: Южная
Laguna De Perlas - nearest hotel: Hotelito Bella Vista;Western Union
Sur - neares

Itoman - nearest hotel: No hotel found
Ibshaway - nearest hotel: No hotel found
Puerto Carreno - nearest hotel: Carreño Plaza
San Luis De La Loma - nearest hotel: No hotel found
Ciudad Lazaro Cardenas - nearest hotel: Hotel Sol del Pacífico
Benavente - nearest hotel: Hostal Universal
Port Blair - nearest hotel: Andaman Ocean Breeze


In [9]:
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Port-Aux-Francais,TF,-49.3500,70.2167,67,Keravel
1,Sabang,ID,5.8933,95.3214,80,Sabang Hill
2,Blackmans Bay,AU,-43.0167,147.3167,78,Villa Howden
3,Saudarkrokur,IS,65.7461,-19.6394,81,Mikligarður
4,Manali,IN,13.1667,80.2667,78,Vijay Park
...,...,...,...,...,...,...
577,Puerto Carreno,CO,6.1890,-67.4859,78,Carreño Plaza
578,San Luis De La Loma,MX,17.2706,-100.8939,74,No hotel found
579,Ciudad Lazaro Cardenas,MX,17.9583,-102.2000,86,Hotel Sol del Pacífico
580,Benavente,ES,42.0025,-5.6783,55,Hostal Universal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
# Define the longitude and latitude limits
min_longitude = -180
max_longitude = 180
min_latitude = -90
max_latitude = 90

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["Hotel Name", "Country"],  
    alpha=0.5,
    geo=True,
    xlim=(min_longitude, max_longitude),
    ylim=(min_latitude, max_latitude),
    tiles="CartoLight",
    title="Hotel Map",
    colorbar=True)

hotel_map.opts(width=500, height=400)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)